---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
# Remove non-states
non_states = ['AS', 'DC', 'GU', 'MP', 'NA', 'PR', 'VI']
for item in non_states:
    del states[item]
#len(states) 
#states

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )'''
    
    # Open and read file
    f = open('university_towns.txt', 'r')
    temp = []
    for row in f.readlines():
        Data = row.split("\n")
        temp.append(Data)
    f.close()    

    flattened_list = [y for x in temp for y in x]
    list2 = [x for x in flattened_list if x != ""]
    #list2

    pos = []  # show where index position is for each state
    for item in states.values():
         x = list2.index(item + "[edit]") 
         pos.append(x)   
    pos.sort()  
    #len(pos) # should be 50 for 50 states
    #pos
    
    # Make 2 parallel lists that show start and stop index positions of regions
    start = []
    for loc in pos:
        start.append(loc + 1)
    del start[-1] # get rid of last    
    start.append(566)   
    stop = []
    for i in pos[1: ]: # get rid of first by starting at index position 1
        a = i-1
        stop.append(a)
    stop.append(567) 
    #start
    #stop
    regions = []  # extract regions by start and stop positions
    for i in range(50):
        x = list2[start[i]:stop[i] + 1]
        regions.append(x)
    #regions.append([list2[-1]])
    #regions

    # sort states in alpha order
    s = list(states.values())
    s.sort()
    #s

    # count how many regions per states
    numstates = []
    for i in regions:
        x = len(i)
        numstates.append(x)
    #sum(numstates)   

    # create a flattened list with the correct number of state entries
    result = [num*[name] for num, name in zip(numstates,s)]
    flattened_states = [val for sublist in result for val in sublist]
    #flattened_states[0:10]
    #flattened_states[-10:]
    #len(flattened_states)
    #flattened_states

    # create a flattened list with the names of regions
    flattened_regions = [val for sublist in regions for val in sublist]
    #flattened_regions[0:10]
    #flattened_regions[-10:]
    #len(flattened_regions)

    r1 = flattened_regions.index('North Mankato, South Central College')
    r2 = flattened_regions.index('The Colleges of Worcester Consortium:')
    r3 = flattened_regions.index('The Five College Region of Western Massachusetts:')
    r4 = flattened_regions.index('Faribault, South Central College')
    #r1, r2, r3, r4 find index positions of entries that need to be altered to fit the pattern

    flattened_regions[223] = 'North Mankato, South Central College ()'
    flattened_regions[179] = 'The Colleges of Worcester Consortium: ()'
    flattened_regions[184] = 'The Five College Region of Western Massachusetts: ()'
    flattened_regions[217] = 'Faribault, South Central College ()'
    #flattened_regions

    reg = []
    for item in flattened_regions:
        if "(" in item:
            x = item.split("(")[0]
            reg.append(x)
    regions_only = []
    for item in reg:
        a = item.rstrip() # strip out the white space at the end of each region
        regions_only.append(a)

    # Make the pandas dataframe
    s1 = pd.Series(flattened_states,  name='State')
    s2 = pd.Series(regions_only, name='RegionName')
    df = pd.concat([s1, s2], axis=1)
      
    #return "ANSWER"
    return df

#get_list_of_university_towns()

In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    xls_file = pd.ExcelFile('gdplev.xls')
    xls_file
    xls_file.sheet_names
    # Load the xls file's Sheet1 as a dataframe
    df = xls_file.parse('Sheet1')
    #df 
    # drop last column
    df.drop(df.columns[[0, 1, 2, 3, 5, 7]], axis=1, inplace=True)

    # delete row below header and above 1st observation
    df.drop(df.index[0: 219], axis=0, inplace=True) # does not include 219 in drop

    # Rename columns
    df.columns = ['Year_Quarter', 'GDP_chained2009_dollars']
    #df.head()

    df.reset_index("index", inplace=True)

    df.drop(df.columns[[0]], axis=1, inplace=True)
    # df.shape

    #df['shift_col1'] = df['GDP_chained2009_dollars'].shift()
    #df['shift_col2'] = df['shift_col1'].shift()
    #df['diff'] = (df['shift_col2'] > df['shift_col1']) & (df['shift_col1'] > df['GDP_chained2009_dollars']) == True
    #df
    #df2 = df[df['diff'] == True]
    #result = df2.iloc[0,0]

    alist = df.GDP_chained2009_dollars.tolist()
    blist = alist.copy()
    del blist[0]
    clist = blist.copy()
    del clist[0:1]

    tally = []
    for val,item, num in zip(alist[0:-2], blist, clist):
       if (val > item): 
            if (item > num):
                tally.append(item)

    ans = tally[0]
    result = df.Year_Quarter.loc[df.GDP_chained2009_dollars == ans].tolist()[0]
    #return "ANSWER"
    return result 

#get_recession_start()  # '2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    xls_file = pd.ExcelFile('gdplev.xls')
    xls_file
    xls_file.sheet_names
    # Load the xls file's Sheet1 as a dataframe
    df = xls_file.parse('Sheet1')
    #df 
    # drop last column
    df.drop(df.columns[[0, 1, 2, 3, 5, 7]], axis=1, inplace=True)

    # delete row below header and above 1st observation
    df.drop(df.index[0: 219], axis=0, inplace=True) # does not include 219 in drop

    # Rename columns
    df.columns = ['Year_Quarter', 'GDP_chained2009_dollars']
    #df.head()

    df.reset_index("index", inplace=True)

    df.drop(df.columns[[0]], axis=1, inplace=True)
    # df.shape

    #df['shift_col1'] = df['GDP_chained2009_dollars'].shift()
    #df['shift_col2'] = df['shift_col1'].shift()
    #df['diff'] = (df['shift_col2'] > df['shift_col1']) & (df['shift_col1'] > df['GDP_chained2009_dollars']) == True
    #df
    #df2 = df[df['diff'] == True]
    #result = df2.iloc[0,0]

    alist = df.GDP_chained2009_dollars.tolist()
    blist = alist.copy()
    del blist[0]
    clist = blist.copy()
    del clist[0:1]

    tally = []
    for val,item, num in zip(alist[0:-2], blist, clist):
       if (val > item): 
            if (item > num):
                tally.append(item)

    ans = tally[0]
    result = df.Year_Quarter.loc[df.GDP_chained2009_dollars == ans].tolist()[0]
    indx = df[df['Year_Quarter'] == result].index.tolist()[0]
    ans = df.GDP_chained2009_dollars[indx + 1: ]
    bottom = ans.min()
    indx2 = df[df['GDP_chained2009_dollars'] == bottom].index.tolist()[0]  # index position 37 for bottom GDP
    recession_end = indx2 + 2
    ans = df.Year_Quarter[indx2 + 2]
    #return "ANSWER"
    return ans

#get_recession_end()   #'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    xls_file = pd.ExcelFile('gdplev.xls')
    xls_file
    xls_file.sheet_names
    # Load the xls file's Sheet1 as a dataframe
    df = xls_file.parse('Sheet1')
    #df 
    # drop last column
    df.drop(df.columns[[0, 1, 2, 3, 5, 7]], axis=1, inplace=True)

    # delete row below header and above 1st observation
    df.drop(df.index[0: 219], axis=0, inplace=True) # does not include 219 in drop

    # Rename columns
    df.columns = ['Year_Quarter', 'GDP_chained2009_dollars']
    #df.head()

    df.reset_index("index", inplace=True)

    df.drop(df.columns[[0]], axis=1, inplace=True)
    # df.shape

    #df['shift_col1'] = df['GDP_chained2009_dollars'].shift()
    #df['shift_col2'] = df['shift_col1'].shift()
    #df['diff'] = (df['shift_col2'] > df['shift_col1']) & (df['shift_col1'] > df['GDP_chained2009_dollars']) == True
    #df
    #df2 = df[df['diff'] == True]
    #result = df2.iloc[0,0]

    alist = df.GDP_chained2009_dollars.tolist()
    blist = alist.copy()
    del blist[0]
    clist = blist.copy()
    del clist[0:1]

    tally = []
    for val,item, num in zip(alist[0:-2], blist, clist):
       if (val > item): 
            if (item > num):
                tally.append(item)

    ans = tally[0]
    result = df.Year_Quarter.loc[df.GDP_chained2009_dollars == ans].tolist()[0]
    indx = df[df['Year_Quarter'] == result].index.tolist()[0]
    ans = df.GDP_chained2009_dollars[indx + 1: ]
    bottom = ans.min()
    indx2 = df[df['GDP_chained2009_dollars'] == bottom].index.tolist()[0]  # index position 37 for bottom GDP
    result = df.Year_Quarter.loc[[indx2]].tolist()[0]
    return result

#get_recession_bottom()  # '2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    
    '''
    df = pd.read_csv("City_Zhvi_AllHomes.csv")
    #df.head()

    # Break up dataframe into 3 parts before recombining
    df_a = df.ix[:,'2000-01':'2016-06']
    df_b = df.ix[:,'2016-07':'2016-08']
    df_c = df.ix[:,'RegionName':'State']

    df_a = df_a.groupby(pd.PeriodIndex(df_a.columns, freq='Q'), axis=1).mean()
    #df_a.head()

    df_b["2016Q3"] = df[['2016-07', '2016-08']].mean(axis=1)
    df_d = df_b[["2016Q3"]]
    #df_d.head()

    df_e = pd.concat([df_c, df_a, df_d], axis=1)
    df_e["State"].replace(states, inplace=True)
    df_e = df_e.set_index(['State', 'RegionName'])
    df_e = df_e.sortlevel(level='State')
    new_cols = []
    for item in df_e.columns:
        x = str(item).lower()
        new_cols.append(x)
    df_e.columns = new_cols
    
    #return "ANSWER"  .loc["Texas"].loc["Austin"].loc["2010q3"]
    return df_e

# convert_housing_data_to_quarters()

In [31]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).
    
    Hypothesis: University towns have their mean housing prices less effected by recessions. 
    Run a t-test to compare the ratio of the mean price of houses in university towns the quarter 
    before the recession starts compared to the recession bottom. 
    (price_ratio=quarter_before_recession/recession_bottom)'''
    
    # start of recession is '2008q3'  the column before the get_recession_start() result so use '2008q2'
    # bottom of recession is  # '2009q2'

    df = convert_housing_data_to_quarters()
    df = df.reset_index()
    df_housing = df[['RegionName','State','2008q2','2009q2']]
    df_housing["price_ratio"] = df_housing["2008q2"]/df_housing["2009q2"]
    
    df_unitowns = get_list_of_university_towns()
    #len(df_unitowns)  # 517
    
    merged_inner = pd.merge(left = df_housing, right=df_unitowns, how='inner', left_on=['State', 'RegionName'], right_on=['State', 'RegionName'])
    merged_outer = pd.merge(left = df_housing, right=df_unitowns, how='outer', left_on=['State', "RegionName"], right_on=['State', "RegionName"])
    merged_outer_subtract_inner=merged_outer[~merged_outer.RegionName.isin(df_unitowns.RegionName)]
    
    univ_towns = merged_inner.dropna(how="any")
    non_towns = merged_outer_subtract_inner.dropna(how="any")
    
    univ_towns.set_index(['State', 'RegionName'], inplace = True)
    non_towns.set_index(['State', 'RegionName'], inplace = True)
    
    #The mean should be taken at the very end, on the "price_ratio" columns in both dataframes. 
    #Don't need to create additional columns with the mean. But you need to create the "price_ratio" 
    #on the quarter-columns selected according to the instructions on the recession.
    
    a = univ_towns.price_ratio.mean()
    b = non_towns.price_ratio.mean()
    

    # compares whether difference found in 2 means, result is a tuple with t-stat and p-value
    
    from scipy import stats

    # compares whether difference found in 2 means, result is a tuple with t-stat and p-value

    x = stats.ttest_ind(univ_towns.price_ratio, non_towns.price_ratio)
    
    if a < b:
        better = "university town"
    else:
        better = "non-university town"
    if x[1] < 0.01:
        different = True  # so reject null hypothesis
    else:
        different = False # no difference between groups so null hypothesis can not be rejected   
    
    #return "ANSWER"
    return (different, x[1], better)

#run_ttest()

# (False, 0.9774322145570149, 'university town')
# (False, 0.92204834451698192, 'university town')
# (False, 0.92204834451698192, 'non-university town')

/Applications/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(True, 0.0019874364602259775, 'university town')